# 머신러닝
- 인공신경망
- 의사결정나무
- RandomForest
- SVM
- 로지스틱 회귀 분석

In [1]:
options(warn = -1)
library(nnet)
library(party)
library(randomForest)
library(e1071)
library(caret)

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘ggplot2’

The following object is masked from ‘package:randomForest’:

    margin



In [2]:
# 샘플링을 위한 인덱스 뽑기
set.seed(1234)
samp <- createDataPartition(iris$Species, p=0.7, list=F)

In [3]:
# 표준화
iris2 <- as.data.frame(scale(iris[,-5]))
iris2$Species <- iris$Species

# 표준화 안하는 데이터(의사결정나무, RandomForest)
data.tr <- iris[samp,]
data.test <- iris[-samp,]
x <- subset(data.test, select = -Species)
y <- data.test$Species

# 표준화 하는 데이터
data.tr.norm <- iris2[samp,]
data.test.norm <- iris2[-samp,]
x.norm <- subset(data.test.norm, select = -Species)
y.norm <- data.test.norm$Species

# 인공 신경망
f.nnet <- function(data.tr, data.test, x, y) {
    model.nnet <- nnet(Species ~ ., data=data.tr, size = 3) 
    pred <- predict(model.nnet, x, type="class")
    return (mean(pred == y))
}

# 의사 결정 나무
f.ctree <- function(data.tr, data.test, x, y) {
    model.ctree <- ctree(Species ~ ., data=data.tr)
    pred <- predict(model.ctree, x)
    return (mean(pred == y))
}

# RandomForest
f.randomForest <- function(data.tr, data.test, x, y) {
    model.rf <- randomForest(Species ~ ., data=data.tr, ntree=100)
    pred <- predict(model.rf, x)
    return (mean(pred == y))
}


# SVM
f.svm <- function(data.tr, data.test, x, y) {
    model.svm <- svm(Species ~., data=data.tr)
    pred <- predict(model.svm, x)
    return (mean(pred == y))
}

# 다항 로지스틱 회귀분석
f.multinom <- function(data.tr, data.test, x, y) {
    model.multinom <- multinom(Species ~ ., data=data.tr)
    pred <- predict(model.multinom, x)
    return (mean(pred == y))
}

In [4]:
result <- rep(0,5)
for (i in 1:5) {
    result[1] <- result[1] + f.nnet(data.tr.norm, data.test.norm, x.norm, y.norm)
    result[2] <- result[2] + f.ctree(data.tr, data.test, x, y)
    result[3] <- result[3] + f.randomForest(data.tr, data.test, x, y)
    result[4] <- result[4] + f.svm(data.tr.norm, data.test.norm, x.norm, y.norm)
    result[5] <- result[5] + f.multinom(data.tr.norm, data.test.norm, x.norm, y.norm)
}
result <- result / 5

# weights:  27
initial  value 124.913043 
iter  10 value 6.777899
iter  20 value 0.018147
final  value 0.000081 
converged
# weights:  18 (10 variable)
initial  value 115.354290 
iter  10 value 7.556637
iter  20 value 0.042347
iter  30 value 0.009210
iter  40 value 0.006761
iter  50 value 0.003904
iter  60 value 0.003512
iter  70 value 0.003095
iter  80 value 0.002826
iter  90 value 0.002781
iter 100 value 0.002705
final  value 0.002705 
stopped after 100 iterations
# weights:  27
initial  value 128.751388 
iter  10 value 25.241454
iter  20 value 0.068786
iter  30 value 0.000395
final  value 0.000081 
converged
# weights:  18 (10 variable)
initial  value 115.354290 
iter  10 value 7.556637
iter  20 value 0.042347
iter  30 value 0.009210
iter  40 value 0.006761
iter  50 value 0.003904
iter  60 value 0.003512
iter  70 value 0.003095
iter  80 value 0.002826
iter  90 value 0.002781
iter 100 value 0.002705
final  value 0.002705 
stopped after 100 iterations
# weights:  27
initial  value 107

In [5]:
result <- data.frame(예측력=result)
rownames(result) <- c("인공신경망", "의사결정나무", "RandomForest", "SVM", "로지스틱회귀분석")

In [6]:
result

,예측력
인공신경망,0.9333333
의사결정나무,0.8888889
RandomForest,0.9288889
SVM,0.9333333
로지스틱회귀분석,0.9333333
